In [ ]:
import pandas as pd
import numpy as np
import re, string, math
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

In [ ]:
train_url="https://raw.githubusercontent.com/sultan-crow/data-science/master/Assignment/Assig2/train3.csv"
#"https://raw.githubusercontent.com/sultan-crow/data-science/master/Assignment/Assig2/train.csv"
test_url="https://raw.githubusercontent.com/sultan-crow/data-science/master/Assignment/Assig2/dev.csv"
#test_url="https://raw.githubusercontent.com/sultan-crow/data-science/master/Assignment/Assig2/train3.csv"

stopWords = stopwords.words("english")

In [4]:
train_df=pd.read_csv(train_url)
train_df.head()

,Insult,Comment
0,0,"""like this if you are a tribe fan"""
1,1,"""you're idiot......................."""
2,1,"""I am a woman Babs, and the only ""war on women..."
3,1,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F..."
4,1,"""haha green me red you now loser whos winning ..."


In [10]:
test_df=pd.read_csv(test_url)
train_df, test_df = train_test_split(test_df)
train_df = train_df.copy()
test_df = test_df.copy()
train_df.head()

,Insult,Comment
600,1,"""U dad already ashamed of u cuz he dnt know if..."
405,1,"""""talk about her how we wish, you are a nobody..."
2167,0,"""get off the nigga dick hot damn"""
977,1,"""Bet he could get your mother?"""
47,0,"""You are so strong"""


In [11]:
reg = re.compile("([a-z ])", re.IGNORECASE | re.DOTALL | re.MULTILINE)
def tokenize(comment_text):
    comment_text = comment_text.lower()
    comment_text = comment_text.translate(comment_text.maketrans(string.punctuation,' ' * len(string.punctuation)))
    comment_text = "".join(reg.findall(comment_text))
    #return comment_text
    return comment_text.split(' ')
def tokenize2(comment_text):
    comment_text = comment_text.lower()
    comment_text = comment_text.translate(comment_text.maketrans(string.punctuation,' ' * len(string.punctuation)))
    comment_text = "".join(reg.findall(comment_text))
    return comment_text
    #return comment_text.split(' ')

  
  
all_words=set()
def build_master_vocab(x):
  all_words.update(word for word in tokenize(x['Comment']) if word not in stopWords)


In [12]:
#Building dictionary/set of all the words in all the documents
train_df.apply(lambda x:build_master_vocab(x), axis=1)
all_words
num_dataset=train_df.shape[0]
num_dataset
#feature_df=pd.DataFrame(index=range(0,num_dataset),columns=all_words).fillna(0)
#feature_df.shape
#train_df.apply(lambda x: build_feature_vector(x['Comment'],x['Insult']))
#train_df.apply(lambda x: tokenize(x['Comment']), axis=1)
#all_words

1676

In [21]:
#print(gradient(Xs,Ys))
def gradient(Xs,Ys,iterations=10000,tolerance=.05,alpha=.2):
  num_dataset,num_features=np.shape(Xs)
  print(num_dataset,num_features)
  thetas=np.ones(num_features).reshape(-1,1)# Initial feature weights as 1
  Xs_T=Xs.T
  print(thetas.shape)
  for i in range(iterations):
    diff=np.dot(Xs,thetas)-Ys
    #print(diff)
    cost=np.dot(diff.T,diff)/(2*num_dataset)
    gradient=np.dot(Xs_T,diff);
    gradient=gradient/num_dataset
    thetas=thetas-alpha*gradient
    #print("Iteration ",i+1,"Cost: ",cost)
    if(cost<tolerance):
      return thetas

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(vocabulary=all_words)
feature_matrix=cv.fit_transform(
    train_df.apply(lambda x: tokenize2(x['Comment']), axis=1)
)

feature_matrix2=cv.fit_transform(
    test_df.apply(lambda x: tokenize2(x['Comment']), axis=1)
)
#print(cv.get_feature_names())
Xs=feature_matrix.toarray()
Ys=train_df['Insult']
Ys=np.array(Ys).reshape(-1,1)
Xs2=feature_matrix2.toarray()

print(Ys.shape)
thetas=gradient(Xs,Ys)
print(thetas.shape,Xs2.shape)
# print(Xs.shape)

(1676, 1)
1676 5987
(5987, 1)
(5987, 1) (559, 5987)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(vocabulary=all_words)
feature_matrix=tfidf.fit_transform(
    train_df.apply(lambda x: tokenize2(x['Comment']), axis=1)
)

feature_matrix2=tfidf.fit_transform(
    test_df.apply(lambda x: tokenize2(x['Comment']), axis=1)
)
#print(cv.get_feature_names())
Xs=feature_matrix.toarray()
Ys=train_df['Insult']
Ys=np.array(Ys).reshape(-1,1)
Xs2=feature_matrix2.toarray()

print(Ys.shape)
thetas=gradient(Xs,Ys)
print(thetas.shape,Xs2.shape)
# print(Xs.shape)

(52, 1)
52 368
(368, 1)
(368, 1) (52, 368)


In [17]:
predictions=np.dot(Xs2,thetas)
#print(predictions)
print(predictions.shape)

(559, 1)


In [18]:
from scipy.stats import logistic
from scipy.special import expit
prob=expit(predictions)
prob[prob > 0.5] = 1
prob[prob <= 0.5] = 0
prob.shape
test_df['prob']=prob
#match=test_df.apply(lambda x: x if x['Insult']==x['prob'],axis=1)
#notmatch=test_df.apply(lambda x: x if x['Insult']!=x['prob'],axis=1)
#notmatch
match=test_df[test_df['Insult']==test_df['prob']]
notmatch=test_df[test_df['Insult']!=test_df['prob']]
print(match.shape,notmatch.shape)

(303, 3) (256, 3)
